In [ ]:
import numpy as np

In [ ]:
class Neuron:
    def __init__(self, value, _children=(), _op='', label=''):
        self.value = value
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children);
        self._op = _op;
        self.label = label # for visualization

    def __repr__(self):
        stringVal = f"{self.value}";
        return stringVal

    def __add__(self,other):
        other = other if isinstance(other, Neuron) else Neuron(other)
        out = Neuron(self.value + other.value, (self, other), '+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __mul__(self,other):
        other = other if isinstance(other, Neuron) else Neuron(other)
        out = Neuron(self.value * other.value, (self, other), '*')
        def _backward():
            self.grad += other.value * out.grad
            other.grad += self.value * out.grad
        out._backward = _backward
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers"
        out = Neuron(self.value**other, (self, ), f'**{other}')
        def _backward():
            self.grad += other * (self.value ** (other - 1)) * out.grad
        out._backward = _backward
        return out

    def __truediv__(self, other):
        other = other if isinstance(other, Neuron) else Neuron(other)
        return self * other**-1

    def exp(self, pos=1):
        x = self.value*pos
        out = Neuron(math.exp(x), (self, ), 'exp')
        def _backward():
            self.grad += out.value * out.grad
        out._backward = _backward
        return out

    def __radd__(self, other): # other + self
        return self + other

    def __rsub__(self, other): # other - self
        return -(self - other)

    def __rmul__(self, other): # other * self
        return self * other

    def __rtruediv__(self, other): # other / self
        return self / other**-1

    def backward(self):
        topology = []
        visited = set()
        def build_topology(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topology(child)
                topology.append(v)
        build_topology(self)

        self.grad = 1.0
        for node in reversed(topology):
            node._backward()

    def __float__(self):
        return float(self.value)
    
    def getGrad(self):
        return float(self.grad)

    def hello(self):
        print("hello")

In [ ]:
def neuronGenerator(generator, *args, **kwargs):
    value = generator(*args, **kwargs)
    return np.vectorize(Neuron)(value)

class Layers:
    layer_selector = []

    def __init__(self,
                 layer_neuron_count,
                 bias_weights,
                 current_iter=0 # DO NOT CHANGE
                 ):
        self.current_iter = current_iter
        self.neuron_count = layer_neuron_count[current_iter]
        self.bias_weight = Neuron(bias_weights[current_iter])
        self.neurons = neuronGenerator(np.zeros,(self.neuron_count))
        Layers.layer_selector.append(self)
        self.next = (
            Layers(layer_neuron_count, bias_weights, current_iter + 1)
            if current_iter < len(layer_neuron_count) - 1
            else None
        )
        self.initWeightUniform()

    def __repr__(self):
        return f"{self.neurons}"

    def initWeightUniform(
            self,
            low=0,
            high=10,
            seed=0
            ):
        if self.next != None:
            rng = np.random.default_rng(seed=seed)
            next_neuron_count = self.next.neuron_count
            self.weight = neuronGenerator(rng.uniform,low,high,size=(next_neuron_count,self.neuron_count))

    def initWeightNormal(
            self,
            mean=5,
            variance=3,
            seed=0
            ):
        if self.next != None:
            rng = np.random.default_rng(seed=seed)
            next_neuron_count = self.next.neuron_count
            self.weight = neuronGenerator(rng.normal,loc=mean,scale=variance*variance,size=(next_neuron_count,self.neuron_count))

    def initWeightZero(
            self
        ):
        if self.next != None:
            next_neuron_count = self.next.neuron_count
            self.weight = neuronGenerator(np.zeros,shape=(next_neuron_count,self.neuron_count))

    def feedforward(self):
        if self.next != None:
            self.next.neurons = np.dot(self.neurons,self.weight.T) + self.bias_weight
            self.next.feedforward()

    def clearOutput(self):
        Layers.layer_selector = []
    


In [ ]:
layers = Layers(
    [3,4,2],
    [0.6,0.7,0.1]
)

# layers.initWeightZero()
# layers.neurons = np.array([3,5,2])

In [ ]:
# print(layers)
# print(layers.weight)
# print(layers.next)
# print(layers.next.weight)
# print(layers.next.next)

# layers.feedforward()

layers.neurons[0]._prev


In [ ]:
np.dot(layers.neurons,layers.weight.T)+layers.bias_weight

# print(layers.bias_weight)